<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">TP : Tables de hachage</h1>

Cette [vidéo de présentation](Fichiers/Tables_de_hachage.mp4) pourra vous aider.

Dans ce devoir, nous allons implémenter une table de hachage qui utilise des listes chaînées pour gérer les collisions.  
L'implémentation ne prendra pas en compte le redimensionnement de la table lorsque le taux de collisions devient trop élevé.

## Structure de ce devoir
Nous travaillerons dans un premier temps sur les fonctions de hachage. Ce sera l'occasion d'implémenter des fonctions de hachage. Nous parlerons brièvement de collisions.

Les tables de hachage, pour gérer les collisions, utilisent des listes chaînées. Il faudra donc les implémenter. Cette partie est relativement facile, et vous permettra, si vous ne l'avez jamais fait, de voir comment on implémente une telle structure de données.

Enfin, nous nous intéresserons à l'implémentation d'une table de hachage à proprement parler : pour ce faire, nous combinerons les deux premières parties. La présence de multiples fonctions de hachage vous permettra de comparer leurs performances relatives : en effet, la fonction de hachage ultime n'existe pas, et nous verrons que certaines donnent lieu à moins de collisions que d'autres.

Pour ce travail, nous conviendrons que les entiers sont des entiers naturels encodés sur 32 bits.

## Des objets et des haches
L'idée est d'illustrer comment fonctionne le principe des tables de hachage. Une table de hachage stocke des **objets** ; ces objets ont eux-mêmes un **hash**, c'est-à-dire une valeur unique qui leur est associée.

Puisque nous voulons comprendre comme fonctionne le mécanisme de hachage, nous n'allons pas réutiliser le mécanisme de hachage standard de Python (la fonction [`hash()`](https://docs.python.org/fr/3/library/functions.html#hash)). Nous allons au contraire définir une classe `Objet`, qui sera la classe des objets allant dans la table de hachage.

Voici la définition de la classe `Objet` :

In [ ]:
class Objet():
    def __init__(self, nom: str):
        self.nom = nom
    def hachage(self):
        raise NotImplementedError

Cette classe est **abstraite**, c'est-à-dire qu'elle n'a pas vocation à être instanciée.

Votre rôle ici sera de créer des classes concrètes qui **héritent** de la classe `Objet` et qui fournissent des implémentations pour les deux méthodes abstraites. Ainsi, vous pourrez créer `Objet1` qui hérite de `Objet`, puis écrire :

```python
o = Objet1()
```

Du point de vue fonctionnel, un `Objet` est simplement une classe qui stocke un nom, et qui offre une méthode pour calculer le hash associé à l'`Objet`.

La question cruciale est : comment comparer des objets ?  En effet, la table de hachage sait que les éléments sont des `Objet`s, et qu'ils ont un champ `nom`, qui permet de comparer un objet à un autre. On casse ainsi les frontières de l'abstraction au bénéfice d'un TD plus simple. Une vraie implémentation se doit d'être **générique**, c'est-à-dire de fonctionner pour n'importe quel type `V` des éléments. Comment fait-on pour comparer des éléments de type `V` alors qu'on n'a, par définition, aucune information sur la nature de `V` ? Il suffit d'exiger que `V` fournisse une méthode `__eq__()` qui implémente la comparaison. C'est vrai de tout objet Python : en effet, chaque classe Python hérite de la classe de base `Object`, qui fournit une fonction `__eq__()`.

L'implémentation par défaut de la fonction [`__eq__()`](https://docs.python.org/fr/3/reference/datamodel.html#object.__eq__) utilise l'adresse de l'objet ([`id()`](https://docs.python.org/fr/3/library/functions.html#id)) : un objet est, par défaut, égal seulement à lui-même. En pratique, on veut faire mieux : si vous avez des listes de personnes, on voudra dire que deux personnes sont égales du moment que les noms, prénoms, et adresses email sont égales. Ainsi, deux objets distincts en mémoire pourront être considérés structurellement égaux.

Pour cette raison, dans la vie réelle, avant de ranger vos `Objet`s dans la table de hachage, vous devrez implémenter une fonction `__eq__()` correcte. 

Parlons maintenant de la relation entre `__eq__(self, other)` et `__hash__(self)`.

La classe `Objet` fournit aussi une implémentation par défaut de [`__hash__()`](https://docs.python.org/fr/3/reference/datamodel.html#object.__hash__), qui se base également sur l'adresse mémoire. Imaginons un instant que nous redéfinissions `__eq__()` sans redéfinir `__hash__()`. Deux objets distincts en mémoire pourraient ainsi être égaux, sans pour autant avoir le même hash. Il serait alors possible d'ajouter chaque objet séparément dans la table de hachage, car ils finiraient dans des listes distinctes. On aurait ainsi violé l'invariant de la table de hachage.

Il est donc essentiel, dès lors que l'on modifie `__eq__()`, de modifier la fonction `__hash__()` de manière à garantir l'invariant suivant : **deux objets égaux ont nécessairement le même hash**.


### Constructeur
Les objets peuvent être construits à l'aide du constructeur `Objet1(String nom)`. Commencez par implémenter ce constructeur.

### Fonction de hash standard
Nous allons d'abord implémenter une fonction de hachage standard. Si `s` est la chaîne de caractères, alors son hachage est défini par :

$$hachage(s) \equiv  \left( \sum_{i=0}^{n-1}s[i] \times 31^{n−1−i} \right) \left[2^{32}\right]$$

ou $s[i]$ est la valeur unicode de la $i-ème$ lettre de $s$.

En d'autres termes,


$$hachage(s) \equiv \left( s[0] \times 31^{n−1}+…+s[n−2] \times 31+s[n−1] \right) \left[2^{32}\right] $$

Cette fonction peut s'implémenter à l'aide d'une simple boucle `for` sans utiliser autre chose que des additions ou des multiplications. Elle renvoie donc la valeur de hachage de l'objet.

Implémentez la méthode `hachage()` de la classe `Objet1`. Nous rappelons que que le $i-ème$ caractère de la chaîne `s` s'obtient avec `s[i]`.  
La fonction `ord()` vous sera également utile.

In [ ]:
class Objet1(Objet):
    def hachage(self):
        pass

Testez votre fonction de hachage.  
La définition ci-dessus est bien définie pour la chaîne vide ! Quel doit être son hash ?

In [ ]:
o1 = Objet1("")
assert o1.hachage() == 0
o2 = Objet1("coucou")
assert o2.hachage() == 2940381024
o3 = Objet1("Quelques tests")
assert o3.hachage() == 3871525502
o4 = Objet1("pour s'assurer")
assert o4.hachage() == 3676260147
o5 = Objet1("que votre fonction")
assert o5.hachage() == 735310627
o6 = Objet1("donne la bonne")
assert o6.hachage() == 4060317683
o7 = Objet1("sortie")
assert o7.hachage() == 3398374202

### Fonction de hash alternative
Il existe tout un folklore de fonctions de hachage : la plupart ont été élaborées de manière empirique, utilisent des constantes qui, en pratique, donnent une bonne distribution, et sont le fruit de beaucoup d'essais / erreurs. Une fonction de hachage doit bien distribuer les bits de son entrée ; une manière classique de le faire est à l'aide d'un processus itératif qui combine une valeur initiale avec les caractères de la chaîne.

Voici une deuxième fonction de hachage, que nous vous proposons d'implémenter. Elle combine le $i-ème$ caractère avec le hash `h` de manière différente.

$$
\left \{
\begin{array}{rcl}
\text{hash}(i)& \equiv & \left( \text{hash}(i−1) \times 33⊕s[i] \right) \left[2^{32}\right] \\
\text{hash}(−1)&=&5381
\end{array}
\right.
$$

L'opérateur mathématique $⊕$ est le « ou exclusif » ; il est disponible en Python via l'opérateur `^`. Cette définition se prête naturellement à un calcul itératif.

Implémentez la classe `Objet2` qui est en tous points similaire à la classe `Objet1`, si ce n'est que sa fonction `hachage` utilise la fonction de hash alternative.

In [ ]:
class Objet2(Objet):
    def hachage(self):
        pass

In [ ]:
o1 = Objet2("")
assert o1.hachage() == 5381
o2 = Objet2("coucou")
assert o2.hachage() == 1544958309
o3 = Objet2("Quelques tests")
assert o3.hachage() == 1981306143
o4 = Objet2("pour s'assurer")
assert o4.hachage() == 3716399800
o5 = Objet2("que votre fonction")
assert o5.hachage() == 4093322886
o6 = Objet2("donne la bonne")
assert o6.hachage() == 143843854
o7 = Objet2("sortie")
assert o7.hachage() == 2170033363

### Fonction de hash fournie
Nous fournissons une classe `Objet3` qui implémente une troisième fonction de hachage.

In [ ]:
class Objet3(Objet):
    def hachage(self):
        h = 0
        for i in range(len(self.nom)):
            ki = ord(self.nom[i])
            h = (h << 4) + ki
            g = h & 0xf0000000
            if g != 0:
                h = h ^ (g >> 24)
                h = h ^ g
        return h % (2 ** 32)

## Listes simplement chaînées
Nous avons maintenant besoin de listes pour stocker les éléments qui possèdent le même hash. Des listes simplement chaînées suffisent. L'objet de cette partie est d'implémenter des listes impératives ; pour ne pas compliquer inutilement le sujet, nos listes ne seront pas génériques : vous pouvez faire l'hypothèse que les éléments de la liste sont des `Objet`s.

### Rappel sur les fonctionnement des listes impératives
Une liste simplement chaînée est constituée d'une successions de cellules ; chaque cellule pointe vers la suivante, et la dernière cellule ne pointe vers rien, c'est-à-dire que son champ `suivant` vaut `None`.

Nous voulons ici une structure de données impérative. Si ma liste s'appelle `l` et que je souhaite y ajouter l'objet `o`, écrire `l.ajouterTete(o)` a pour effet de modifier `l` **en place**. Après cette ligne, `l` a changé et contient désormais `o`.

Les structures de données impératives s'opposent aux structures fonctionnelles, ou persistentes : une autre approche aurait consisté à renvoyer une nouvelle liste. Ainsi, on aurait pu écrire `l2 = l.ajouterTete(o)`. Dans ce cas-là, `l2` aurait contenu la nouvelle liste, tandis que `l` aurait toujours contenu l'ancienne liste.

Pour implémenter facilement une liste impérative, on peut créer, en plus de la classe `Cellule`, une classe `Liste`, qui contient une référence vers la tête de la liste. Ainsi, `Liste` aura un champ `tete`. Ce champ sera modifié au fur et à mesure des appels à `ajouterTete` et `supprimerTete` : c'est donc bien une structure de données impératives.

Nous imposons dans ce sujet une contrainte supplémentaire : les fonctions d'ajout et de suppression **renvoient l'objet liste lui-même**. Ceci permet de chaîner les appels, comme dans par exemple `l.ajouterTete(o1).ajouterTete(o2)`.

### Implémentation attendue
Vous êtes libre de choisir la représentation que vous voulez pour les cellules. En revanche, vos listes chaînées doivent implémenter la classe appelée `Liste` et offrir les fonctions suivantes :

* `Liste()`, le constructeur par défaut;
* `ajouteTete(val)`, qui ajoute `val` en tête de la liste, et renvoie la liste elle-même ;
* `supprimeTete()`, qui lance une exception `NoSuchElementException` si la liste est vide, ou supprime l'élément en tête de la liste, et renvoie la liste elle-même sinon ;
* `contient(o)`, qui renvoie `True` si la liste contient un élément portant le même nom que `o`
* `longueur()` qui renvoie la longueur de la liste.

### Tests
Vérifiez bien que vos fonctions ont la sémantique attendue. N'oubliez pas que le code suivante doit réussir, car on compare les objets de manière **structurelle** : c'est le nom des objets qui est comparé, et pas l'adresse de l'objet. Votre code de `Liste` doit donc tenir compte de ce fait.

In [ ]:
class NoSuchElementException(Exception):
    """ Definit l'exception NoSuchElementException"""

class Cellule:
    pass

class Liste:
    pass

In [ ]:
l = Liste()
l.ajouteTete(Objet1("toto"));
assert l.contient(Objet1("toto")) == True

In [ ]:
l = Liste()
o = Objet2("1")
l.ajouteTete(o)
assert l.longueur() == 1
l.ajouteTete(Objet2("2")).ajouteTete(Objet2("3"))
assert l.contient(o) == True
assert l.longueur() == 3
assert l.contient(Objet2("4")) == False 
assert l.contient(Objet2("2")) == True
assert l.supprimeTete().longueur() == 2
assert l.contient(Objet2("3")) == False
assert l.contient(o) == True
try: 
    l.supprimeTete()
    assert l.longueur() == 1
    l.supprimeTete()
    assert l.longueur() == 0
    l.supprimeTete()
except NoSuchElementException:
    assert l.longueur() == 0
l.ajouteTete(Objet2("5"))
assert l.longueur() == 1
l.contient(Objet2("5")) == True

## Tables de hachage
Il s'agit maintenant de combiner le travail effectué dans les deux premières parties pour implémenter des tables de hachage.

La table de hachage contiendra un tableau de `Liste`s à usage interne, et fera appel aux méthodes `hachage()` des `Objet`s stockés. Encore une fois, la table de hachage ne sera pas générique : vous pouvez donc faire l'hypothèse que tous les objets que vous manipulez sont des `Objet`s. Ceci sera particulièrement utile lors de la gestion des collisions.

Voici la description de la classe `TableDeHachage` que vous devez écrire.

* `TableDeHachage(n)`, constructeur qui prend la taille initiale de la table (nombre de listes utilisées en interne).
* `ajoute(o)`, pour ajouter un élément dans la table à la position `o.hachage() % n`.
* `contient(o)`, pour tester si la table contient un élément.
Nous vous demandons d'écrire une fonction supplémentaire par rapport à l'interface standard des tables de hachage. Cette fonction nous permettra de tester que votre code se comporte correctement.
* `remplissageMax()`: cette fonction trouve la `Liste` qui est la plus remplie, et renvoie un tableau de deux entiers. Le premier entier est l'index de cette liste dans le tableau interne, et le second est le nombre d'éléments dans la liste. Dans le cas où plusieurs listes ont le même remplissage, cette fonction doit renvoyer la "première" liste, c'est-à-dire celle dont l'index est le plus petit dans la table. Dit encore différemment, celle qui a le plus petit hash après modulo.

Comparez les remplissages maximaux pour les `Objet1`, `Objet2` et `Objet3`.  
Quelle est la meilleure fonction de hachage ?

In [ ]:
def prng(n):
    """Pseudo RaNdom Generator"""
    seed = 1
    m = 2147483647
    a = 16807
    b = 0
    for _ in range(n):
        seed = (seed * a + b) % m
    return seed

In [ ]:
t = TableDeHachage(3000)
for i in range(1500):
    t.ajoute(Objet3("chaine" + str(prng(i))))
assert t.contient(Objet3("")) == False
assert t.contient(Objet3("chaine")) == False
assert t.contient(Objet3("eniach")) == False
assert t.contient(Objet3("chaine877819790")) == True
assert t.contient(Objet3("chaine878197790")) == False
assert t.contient(Objet3("chaine1517273377")) == True
assert t.contient(Objet3("chaine1172753377")) == False
assert t.contient(Objet3("chaine1462863342")) == True
assert t.contient(Objet3("chaine1628643342")) == False
assert t.contient(Objet3("chaine1715469037")) == False
remplissage = t.remplissageMax()
assert len(remplissage) == 2
assert remplissage[0] == 848
assert remplissage[1] == 5

Testez votre code avec une fonction de hachage idiote, qui renvoie tout le temps la même valeur. Vous vous retrouverez avec la pire complexité, mais vous pourrez tester facilement votre gestion des collisions.

Vérifiez que votre table de hachage a bien le comportement attendu sur ce test.  
Les deux chaînes utilisées n'ont pas été choisies au hasard. Quelle est leur particularité ?

In [ ]:
t = TableDeHachage(10)
t.ajoute(Objet1("FB"))
t.contient(Objet1("Ea")) == False

## Questions facultatives
Voici quelques questions bonus.

* Implémentez une opération de suppression d'un élément dans la table de hachage. Vous devrez implémenter la suppression en place d'un élément dans une liste.
* Implémentez le redimensionnement de la table. Commencez par garder un compteur du nombre d'éléments dans la table. À chaque ajout, si le taux de remplissage dépasse 0.5, créez un tableau interne deux fois plus grand, et ajoutez de nouveau tous les éléments dans le tableau.

## Source :
Coursera, *Conception et mise en œuvre d'algorithmes*, Ecole Polytechnique